# Analisis de informacion de Grupo Interact

##### Conexion a base de datos MySQL para obtener informacion almacenada

In [66]:
import pandas as pd
from sqlalchemy import create_engine

localhost = 'localhost'
user = 'root'
password = 'Mysql12345'
database = 'reportehoras'

# Configurar conexión y probarla

In [67]:
def cargar_tabla(nombre_tabla, con, chunk_size=1000):
    """
    Carga tabla en un DF usando chunks para optimizar la lectura.
    :param nombre_tabla: # Nombre de la tabla
    :param con: # Conexion a la base de datos
    :param chunk_size: # tamano de los pedazos
    :return: Object tipo DataFrame
    """
    return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)

try:
    con = create_engine(f'mysql+pymysql://{user}:{password}@{localhost}/{database}')
    if con is not None:      
        
        tablas = ["clasificaciones", "clientes", "datos_act", "empleados", "modalidades"]
        dfs = {tabla: cargar_tabla(tabla, con) for tabla in tablas}
        
        df_clasificaciones = dfs['clasificaciones']
        df_clientes = dfs['clientes']
        df_datos_act = dfs['datos_act']
        df_empleados = dfs['empleados']
        df_modalidades = dfs['modalidades']
                
except RuntimeError as e:
    print(f'❌ Contraseña incorrecta')
    con = None
except Exception as err:
    print(f"❌ Error de conexión: {err}")
    con = None
finally:
    if con is not None:
        con.dispose()

C:\Users\variz\AppData\Local\Temp\ipykernel_9548\181994467.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)


##### Realizar merge para obtener df con variables categoricas de llaves foraneas

In [49]:
df_datos_acts.head()

,clave_act,numero,clave_cli,clave_mod,clave_cla,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion,bloqueado,tipo,horas,fecha_capt,hora_capt,usuario,baja,tiempo_tot,tiempo_desc
0,1,9011,18,1,22,2024-05-01,07:00,10:00,0.0,0.0,*Reunión para planeación del día con los compa...,1.0,A,None,2024-10-08,10:59:26,9030,0,3.0,3.0
1,2,9011,25,1,45,2024-05-01,10:00,10:30,0.0,0.0,*Soporte a A. Chávez en pólizas entregadas de ...,1.0,A,None,2024-10-08,10:59:26,9030,0,0.5,0.5
2,3,9034,18,1,22,2024-05-01,08:00,09:00,0.0,0.0,capacitación 5s,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
3,4,9034,18,1,22,2024-05-01,09:00,10:00,0.0,0.0,Capacitacion proyecto de reporte de horas,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
4,5,9010,18,1,3,2024-05-01,07:00,16:00,0.0,1.0,"Revisión de temas de red nueva oficina, ubicac...",1.0,A,None,2024-06-11,08:57:08,9034,0,9.0,8.0


In [68]:
df_concentrado = pd.merge(df_datos_act, df_clientes[['clave_cli', 'nombre']], how='left', on='clave_cli', suffixes=('_datos','_cli'))
df_concentrado = df_concentrado.merge(df_modalidades[['clave_mod','descripcion']], how='left', on='clave_mod', suffixes=('_total','_modalidades'))
df_concentrado = df_concentrado.merge(df_clasificaciones[['clave_cla','descripcion']], how='left', on='clave_cla', suffixes=('_total', '_clasificaciones'))
df_concentrado = df_concentrado.merge(df_empleados[['numero', 'nombre']], how='left', on='numero', suffixes=('_total', '_empleados'))
df_concentrado.drop(columns=['clave_cli', 'clave_mod', 'clave_cla', 'numero', 'usuario', 'bloqueado', 'horas'], inplace=True)

df_concentrado = df_concentrado[['clave_act', 'nombre_empleados', 'nombre_total', 'descripcion_modalidades', 'descripcion', 'fecha_act', 'hora_ini', 'hora_fin', 'tiempo_t', 'tiempo_p', 'descripcion_total', 'tipo', 'fecha_capt', 'hora_capt', 'tiempo_tot', 'tiempo_desc']]

df_concentrado.head()


,clave_act,nombre_empleados,nombre_total,descripcion_modalidades,descripcion,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion_total,tipo,fecha_capt,hora_capt,tiempo_tot,tiempo_desc
0,1,ROSA MARIA MARTINEZ RAMIREZ,GRUPO INTERACT SA DE CV,Presencial Oficina,Juntas y Reuniones,2024-05-01,07:00,10:00,0.0,0.0,*Reunión para planeación del día con los compa...,A,2024-10-08,10:59:26,3.0,3.0
1,2,ROSA MARIA MARTINEZ RAMIREZ,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,Presencial Oficina,Soporte,2024-05-01,10:00,10:30,0.0,0.0,*Soporte a A. Chávez en pólizas entregadas de ...,A,2024-10-08,10:59:26,0.5,0.5
2,3,ANA CERVANTES PIEDRAS,GRUPO INTERACT SA DE CV,Presencial Oficina,Juntas y Reuniones,2024-05-01,08:00,09:00,0.0,0.0,capacitación 5s,A,2024-06-11,08:55:45,1.0,1.0
3,4,ANA CERVANTES PIEDRAS,GRUPO INTERACT SA DE CV,Presencial Oficina,Juntas y Reuniones,2024-05-01,09:00,10:00,0.0,0.0,Capacitacion proyecto de reporte de horas,A,2024-06-11,08:55:45,1.0,1.0
4,5,NATALY ALCANTARA HERNANDEZ,GRUPO INTERACT SA DE CV,Presencial Oficina,Actividades Administrativas,2024-05-01,07:00,16:00,0.0,1.0,"Revisión de temas de red nueva oficina, ubicac...",A,2024-06-11,08:57:08,9.0,8.0
